In [1]:
!pip install gdown
import gdown
# Replace 'FILE_ID' with the actual ID from your Drive share link
file_id = '1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'augmented_pepper_dataset.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj
From (redirected): https://drive.google.com/uc?id=1yZBuHL9CBjOZjfen8FSas7ouPVQz-yYj&confirm=t&uuid=ee8153f3-7443-4bca-bc9e-176097c206a8
To: /kaggle/working/augmented_pepper_dataset.zip
100%|██████████| 4.00G/4.00G [00:27<00:00, 145MB/s] 


'augmented_pepper_dataset.zip'

In [2]:
import os
import zipfile
import shutil

zip_path = 'augmented_pepper_dataset.zip'
extract_to = '/kaggle/working/training_data'

# 1. Force a clean start by removing the old folder if it exists
if os.path.exists(extract_to):
    shutil.rmtree(extract_to)
    print(f"🗑️ Cleaned up existing folder: {extract_to}")

# 2. Extract the data
print("Extracting augmented dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print("✅ Extraction complete.")

Extracting augmented dataset...
✅ Extraction complete.


In [3]:
import os

# This will list everything in your working directory
for root, dirs, files in os.walk('/kaggle/working/'):
    level = root.replace('/kaggle/working/', '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")

/
.virtual_documents/
training_data/
    content/
        dataset/
            UNHEALTHY/
                Slow-Decline/
                augmented/
                black_pepper_yellow_mottle_virus/
                MERGED/
                black_pepper_leaf_blight/
                Footrot/
                Pollu_Disease/
            HEALTHY/
                augmented/
                Black pepper detection1.v1i.yolov8/
                    test/
                        labels/
                        images/
                    valid/
                        labels/
                        images/
                    train/
                        labels/
                        images/
                black_pepper_healthy/
                MERGED/
                black pepper RAW/


In [4]:

!pip install split-folders ultralytics timm -q

import os
import cv2
import torch
import shutil
import splitfolders
import numpy as np
import timm
from ultralytics import YOLO
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.auto import tqdm

# --- 1. CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RAW_DATA_PATH = '/kaggle/working/training_data/content/dataset'
CROP_DIR = '/kaggle/working/cropped_leaves'
SPLIT_DIR = '/kaggle/working/final_split'
IMG_SIZE = 384 
BATCH_SIZE = 32 # Optimized for P100 16GB VRAM
EPOCHS = 5

# --- 2. STEP 1: AUTO-CROP LEAVES (Detect then Train) ---
def detect_and_crop_dataset():
    if os.path.exists(CROP_DIR):
        print("✅ Cropped directory already exists. Skipping detection...")
        return
    
    detector = YOLO('yolov8n.pt') 
    classes = ['HEALTHY/augmented', 'UNHEALTHY/augmented']
    
    for cls in classes:
        src_path = os.path.join(RAW_DATA_PATH, cls)
        class_label = cls.split('/')[0]
        dst_path = os.path.join(CROP_DIR, class_label)
        os.makedirs(dst_path, exist_ok=True)

        for img_name in tqdm(os.listdir(src_path), desc=f"🎯 Cropping {class_label}"):
            img_path = os.path.join(src_path, img_name)
            img = cv2.imread(img_path)
            if img is None: continue
            
            results = detector(img, conf=0.25, verbose=False)
            if len(results[0].boxes) > 0:
                box = results[0].boxes.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = box
                leaf_crop = img[y1:y2, x1:x2]
                if leaf_crop.size > 0:
                    cv2.imwrite(os.path.join(dst_path, img_name), leaf_crop)
            else:
                shutil.copy(img_path, os.path.join(dst_path, img_name))

detect_and_crop_dataset()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.7 MB/s eta 0:00:0000:01


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


🎯 Cropping HEALTHY:   0%|          | 0/15216 [00:00<?, ?it/s]

🎯 Cropping UNHEALTHY:   0%|          | 0/32736 [00:00<?, ?it/s]

In [5]:
# --- 3. STEP 2: SHUFFLE & SPLIT (80/10/10) ---
if not os.path.exists(SPLIT_DIR):
    print("\n🔀 Shuffling and splitting data (80% Train, 10% Val, 10% Test)...")
    splitfolders.ratio(CROP_DIR, output=SPLIT_DIR, seed=42, ratio=(.8, .1, .1))

# --- 4. STEP 3: PREPARE DATALOADERS (Optimized for P100) ---
transform = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

train_ds = datasets.ImageFolder(os.path.join(SPLIT_DIR, 'train'), transform['train'])
val_ds = datasets.ImageFolder(os.path.join(SPLIT_DIR, 'val'), transform['val_test'])
test_ds = datasets.ImageFolder(os.path.join(SPLIT_DIR, 'test'), transform['val_test'])

# num_workers=2 is best for Kaggle's 2-vCPU limit
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# --- 5. STEP 4: MODEL & TRAINING ---
print(f"\n🔬 Training ViT-Base on leaf details...")
model = timm.create_model('vit_base_patch16_384', pretrained=True, num_classes=2).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler('cuda') 

best_val_acc = 0.0

for epoch in range(EPOCHS):
    # --- TRAINING PHASE ---
    model.train()
    train_correct, train_total = 0, 0
    pbar = tqdm(train_loader, desc=f"🚀 Epoch {epoch+1} [Train]")
    
    for imgs, labels in pbar:
        imgs, labels = imgs.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
        
        with torch.amp.autocast('cuda'):
            outputs = model(imgs)
            loss = criterion(outputs, labels)
        
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        preds = outputs.argmax(dim=1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)
        pbar.set_postfix(acc=f"{100 * train_correct / train_total:.2f}%")

    # --- VALIDATION PHASE ---
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            val_correct += (model(imgs).argmax(1) == labels).sum().item()
            val_total += labels.size(0)
    
    val_acc = 100 * val_correct / val_total
    print(f"📊 Epoch {epoch+1} Summary: Train Acc: {100*train_correct/train_total:.2f}% | Val Acc: {val_acc:.2f}%")
    
    # --- SAVE PARAMETERS AFTER EACH EPOCH ---
    epoch_file = f'pepperai_epoch{epoch+1}.pth'
    torch.save(model.state_dict(), epoch_file)
    
    # Save best model separately
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'pepperai_best_model.pth')
        print(f"🌟 New best model found and saved!")

# --- 6. STEP 5: FINAL TESTING ---
print("\n🏁 Final Testing on 10% Unseen Data...")
model.load_state_dict(torch.load('pepperai_best_model.pth'))
model.eval()
test_correct = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        test_correct += (model(imgs).argmax(1) == labels).sum().item()

print(f"✅ Final Test Accuracy: {100 * test_correct / len(test_ds):.2f}%")


🔀 Shuffling and splitting data (80% Train, 10% Val, 10% Test)...


Copying files: 47952 files [00:09, 5075.02 files/s]



🔬 Training ViT-Base on leaf details...


model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

🚀 Epoch 1 [Train]:   0%|          | 0/1199 [00:00<?, ?it/s]

...

🚀 Epoch 4 [Train]:   0%|          | 0/1199 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b55259e9940>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
     Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b55259e9940> 
   Traceback (most recent call last):
   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
     ^self._shutdown_workers()^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    ^if w.is_alive():
 ^ ^    ^ ^^^^^^^^^^^^^^^^^^^

📊 Epoch 4 Summary: Train Acc: 99.94% | Val Acc: 99.77%


🚀 Epoch 5 [Train]:   0%|          | 0/1199 [00:00<?, ?it/s]

📊 Epoch 5 Summary: Train Acc: 99.93% | Val Acc: 100.00%
🌟 New best model found and saved!

🏁 Final Testing on 10% Unseen Data...
✅ Final Test Accuracy: 99.92%
